# DCGANs

## referência

- https://github.com/NSTiwari/DCGANs-using-Keras-and-TensorFlow


### Objetivos
- Implementar uma DCGAN para geração de faces humanas
- Treinar o modelo utilizando o dataset CelebA
- Avaliar e gerar novas faces sintéticas

## Importação das Bibliotecas

In [ ]:
# Bibliotecas essenciais para o projeto
import os
import numpy as np
import matplotlib.pyplot as plt
import glob

from tqdm.notebook import tqdm
from PIL import Image

# TensorFlow e Keras para Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, array_to_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from google.colab import files

## Download e Preparação do Dataset CelebA

### Sobre o Dataset CelebA
- **Nome**: CelebFaces Attributes Dataset
- **Tamanho**: Mais de 200.000 imagens de celebridades
- **Resolução**: 178×218 pixels
- **Características**: Diversidade étnica, idades e expressões variadas

In [ ]:
# # Instalar API do Kaggle
# !pip install -q kaggle
# !pip install -q kaggle-cli    

In [ ]:
# CONFIGURE SUAS CREDENCIAIS DO KAGGLE
os.environ['KAGGLE_USERNAME'] = "YOUR_KAGGLE_USERNAME" 
os.environ['KAGGLE_KEY'] = "YOUR_KAGGLE_KEY"

In [ ]:
# Download do dataset CelebA do Kaggle
!kaggle datasets download -d jessicali9530/celeba-dataset --unzip



### 4.1 Análise do Dataset

In [ ]:
# Configurar diretório base das imagens
BASE_DIR = '/content/img_align_celeba/img_align_celeba'

# Obter caminhos de todas as imagens
image_paths = []
for img_name in os.listdir(BASE_DIR):
    image_path = os.path.join(BASE_DIR, img_name)
    image_paths.append(image_path)

print(f"Total de imagens encontradas: {len(image_paths):,}")

### 4.2 Redução do Dataset

📝 **Justificativa**: O dataset CelebA tem ~202K imagens. Para fins didáticos e limitações computacionais, vamos usar um subconjunto menor para acelerar o treinamento.

In [ ]:
# Remover 180.000 imagens para reduzir o dataset
SUBSET_SIZE = 180000 
imgs_delete = os.listdir('/content/img_align_celeba/img_align_celeba')[:SUBSET_SIZE]

print("Removendo imagens em excesso...")
for file_ in imgs_delete:
    os.remove(os.path.join('/content/img_align_celeba/img_align_celeba', file_))

# Atualizar lista de caminhos das imagens restantes
image_paths = []
for img_name in os.listdir(BASE_DIR):
    image_path = os.path.join(BASE_DIR, img_name)
    image_paths.append(image_path)

print(f"Imagens restantes: {len(image_paths):,}")

### 4.3 Redimensionamento para 64x64

#### Por que 64x64 pixels?
- **Eficiência computacional**: Menor resolução = treinamento mais rápido
- **Padrão DCGAN**: Facilita o design das camadas (potências de 2)
- **Estabilidade**: GANs funcionam melhor com resoluções moderadas
- **Recursos limitados**: Adequado para ambientes com pouca memória

In [ ]:
# Criar diretório para imagens redimensionadas
%mkdir /content/resized

# Redimensionar todas as imagens para 64x64 pixels
image_list = []
resized_images = []

# Carregar imagens originais
for filename in glob.glob('/content/img_align_celeba/img_align_celeba/*.jpg'):
    img = Image.open(filename)
    image_list.append(img)

# Redimensionar para 64x64
for image in image_list:
    image = image.resize((64, 64))
    resized_images.append(image)

# Salvar imagens redimensionadas
for (i, new) in enumerate(resized_images):
    new.save('{}{}{}'.format('/content/resized/', i+1, '.jpg'))

print(f"{len(resized_images)} imagens redimensionadas e salvas!")

In [ ]:
# Atualizar para diretório de imagens redimensionadas
BASE_DIR = '/content/resized'

# Obter novos caminhos das imagens redimensionadas
image_paths = []
for img_name in os.listdir(BASE_DIR):
    image_path = os.path.join(BASE_DIR, img_name)
    image_paths.append(image_path)

print(f"Imagens redimensionadas disponíveis: {len(image_paths):,}")

In [ ]:
# Visualizar algumas imagens do dataset
plt.figure(figsize=(10, 10))
temp_images = image_paths[:25]
index = 1

for image_path in temp_images:
    plt.subplot(5, 5, index)
    img = load_img(image_path)
    img = np.array(img)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Img {index}', fontsize=8)
    index += 1

plt.suptitle('Dataset CelebA - Amostras Redimensionadas (64x64)', fontsize=14)
plt.tight_layout()
plt.show()

### 4.4 Normalização dos Dados

**Importância da Normalização**:
- O gerador usa ativação `tanh` que produz valores em [-1, 1]
- Necessário normalizar as imagens para o mesmo intervalo
- Fórmula: `(pixel - 127.5) / 127.5`

In [ ]:
# Carregar imagens em array NumPy
print("Carregando imagens para array NumPy...")
train_images = [np.array(load_img(path)) for path in tqdm(image_paths)]
train_images = np.array(train_images)

print(f"Formato do array: {train_images.shape}")

In [ ]:
# Garantir formato correto e tipo float32
train_images = train_images.reshape(train_images.shape[0], 64, 64, 3).astype('float32')

print(f"Array reformatado: {train_images.shape}")

In [ ]:
# Normalizar imagens para [-1, 1]
# Compatível com ativação tanh do gerador
train_images = (train_images - 127.5) / 127.5

print("Normalização concluída!")
print(f"Valor mínimo: {train_images.min():.3f}")
print(f"Valor máximo: {train_images.max():.3f}")

## Arquitetura das Redes

### Hiperparâmetros 

In [ ]:

LATENT_DIM = 100  # Dimensão do ruído de entrada
WEIGHT_INIT = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)  # Inicialização dos pesos
CHANNELS = 3  # Canais RGB


### Arquitetura do Gerador

**Evolução das Dimensões**:
```
Ruído 100D → Dense → 8×8×512 → 16×16×256 → 32×32×128 → 64×64×64 → 64×64×3
```

**Componentes**:
- Dense + ReLU + Reshape
- Conv2DTranspose (upsampling) + ReLU
- Conv2D final com ativação Tanh

In [ ]:
# Construir modelo Gerador (ARQUITETURA ORIGINAL)
model = Sequential(name='generator')

# Camada densa: ruído 1D → representação 3D
model.add(layers.Dense(8 * 8 * 512, input_dim=LATENT_DIM))
model.add(layers.ReLU())

# Converter 1D para formato 3D
model.add(layers.Reshape((8, 8, 512)))

# Upsampling para 16x16
model.add(layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=WEIGHT_INIT))
model.add(layers.ReLU())

# Upsampling para 32x32
model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=WEIGHT_INIT))
model.add(layers.ReLU())

# Upsampling para 64x64
model.add(layers.Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=WEIGHT_INIT))
model.add(layers.ReLU())

# Camada final: produzir imagem RGB
model.add(layers.Conv2D(CHANNELS, (4, 4), padding='same', activation='tanh'))

generator = model
print("Gerador construído:")
generator.summary()

### 5.3 Arquitetura do Discriminador

**Evolução das Dimensões**:
```
Imagem 64×64×3 → 32×32×64 → 16×16×128 → 8×8×64 → Classificação
```

**Componentes**:
- Conv2D + BatchNorm + LeakyReLU
- Dropout para regularização
- Dense final com Sigmoid

In [ ]:
# Construir modelo Discriminador
model = Sequential(name='discriminator')
input_shape = (64, 64, 3)
alpha = 0.2

# Primeira camada convolucional
model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=alpha))

# Segunda camada convolucional
model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=alpha))

# Terceira camada convolucional
model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=alpha))

# Flatten e regularização
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))  # Evita discriminador muito forte

# Camada de saída: classificação real/falso
model.add(layers.Dense(1, activation='sigmoid'))

discriminator = model
print("Discriminador construído:")
discriminator.summary()

## Implementação da DCGAN

### Classe DCGAN

In [ ]:
# Classe DCGAN original
class DCGAN(keras.Model):
    def __init__(self, generator, discriminator, latent_dim):
        super().__init__()
        
        # Modelos componentes
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = LATENT_DIM
        
        # Métricas para monitoramento
        self.g_loss_metric = keras.metrics.Mean(name='g_loss')
        self.d_loss_metric = keras.metrics.Mean(name='d_loss')

    @property
    def metrics(self):
        return [self.g_loss_metric, self.d_loss_metric]

    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(DCGAN, self).compile() # Chamada ao método compile da superclasse
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        # Obter tamanho do batch
        batch_size = tf.shape(real_images)[0]
        
        # Gerar ruído aleatório
        random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))

        # === TREINAR DISCRIMINADOR ===
        with tf.GradientTape() as tape:
            # Perda com imagens reais
            pred_real = self.discriminator(real_images, training=True)
            
            # Labels para imagens reais (com ruído para estabilidade)
            real_labels = tf.ones((batch_size, 1))
            real_labels += 0.05 * tf.random.uniform(tf.shape(real_labels))
            d_loss_real = self.loss_fn(real_labels, pred_real)

            # Perda com imagens falsas
            fake_images = self.generator(random_noise)
            pred_fake = self.discriminator(fake_images, training=True)
            
            fake_labels = tf.zeros((batch_size, 1))
            d_loss_fake = self.loss_fn(fake_labels, pred_fake)

            # Perda total do discriminador
            d_loss = (d_loss_real + d_loss_fake) / 2

        # Atualizar discriminador
        gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))

        # === TREINAR GERADOR ===
        labels = tf.ones((batch_size, 1))
        
        with tf.GradientTape() as tape:
            # Gerar imagens falsas
            fake_images = self.generator(random_noise, training=True)
            pred_fake = self.discriminator(fake_images, training=True)
            
            # Perda do gerador (quer enganar discriminador)
            g_loss = self.loss_fn(labels, pred_fake)

        # Atualizar gerador
        gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))

        # Atualizar métricas
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {'d_loss': self.d_loss_metric.result(), 
                'g_loss': self.g_loss_metric.result()
        }
        

### 6.2 Monitor de Treinamento

In [ ]:
# Classe para monitorar treinamento visualmente
class DCGANMonitor(keras.callbacks.Callback):
    def __init__(self, num_imgs=25, latent_dim=100):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        
        # Ruído fixo para consistência visual
        self.noise = tf.random.normal([25, latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        # Gerar imagens do gerador
        g_img = self.model.generator(self.noise)
        
        # Desnormalizar para visualização
        g_img = (g_img * 127.5) + 127.5
        g_img.numpy()

        # Plotar resultados
        fig = plt.figure(figsize=(8,8))
        for i in range(self.num_imgs):
            plt.subplot(5, 5, i+1)
            img = array_to_img(g_img[i])
            plt.imshow(img)
            plt.axis('off')
        
        plt.suptitle(f'Época {epoch + 1} - Faces Geradas', fontsize=14)
        plt.savefig('/content/generated/epoch_{:03d}.png'.format(epoch))
        plt.show()
        
        # Exibir métricas
        if logs:
            print(f"Época {epoch + 1}:")
            print(f"Perda Gerador: {logs['g_loss']:.4f}")
            print(f"Perda Discriminador: {logs['d_loss']:.4f}")

    def on_train_end(self, logs=None):
        # Salvar modelo gerador
        self.model.generator.save('generator.h5')
        print("Gerador salvo como 'generator.h5'")

print("👁️ Monitor de treinamento configurado!")

In [ ]:
# Criar diretório para imagens geradas
%mkdir /content/generated

## Treinamento da DCGAN

### Configuração dos Otimizadores

In [ ]:
# Criar instância da DCGAN
dcgan = DCGAN(generator=generator, discriminator=discriminator, latent_dim=LATENT_DIM)

In [ ]:
# Configurar otimizadores
D_LR = 0.0001  # Taxa de aprendizado do discriminador
G_LR = 0.0003  # Taxa de aprendizado do gerador (mais rápido)

dcgan.compile(
    g_optimizer=Adam(learning_rate=G_LR, beta_1=0.5),
    d_optimizer=Adam(learning_rate=D_LR, beta_1=0.5),
    loss_fn=BinaryCrossentropy()
)

### 7.2 Início do Treinamento

⏱️ **Tempo estimado**: 30-60 minutos (dependendo da GPU)

📊 **O que observar**:
- Perda do gerador diminuindo gradualmente
- Perda do discriminador estabilizando em ~0.5
- Qualidade visual das faces melhorando progressivamente

In [ ]:
# Iniciar treinamento (PARÂMETROS ORIGINAIS)
N_EPOCHS = 50

# Treinar modelo
history = dcgan.fit(
    train_images, 
    epochs=N_EPOCHS, 
    callbacks=[DCGANMonitor()]
    )

## Avaliação e Geração de Resultados

### Análise das Métricas de Treinamento

In [ ]:
def plot_training_history(history):
    """Plota o histórico de treinamento"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Perda do gerador
    ax1.plot(history.history['g_loss'], label='Gerador', color='blue')
    ax1.set_title('Perda do Gerador')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Perda')
    ax1.legend()
    ax1.grid(True)
    
    # Perda do discriminador
    ax2.plot(history.history['d_loss'], label='Discriminador', color='red')
    ax2.set_title('Perda do Discriminador')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Perda')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plotar histórico
plot_training_history(history)

## Geração de Novas Faces

### Gerar Nova Face Individual

In [ ]:
# Gerar uma nova face humana

# Criar ruído aleatório
noise = tf.random.normal([1, 100])

# Gerar imagem usando gerador treinado
g_img = dcgan.generator(noise)

# Desnormalizar para visualização
g_img = (g_img * 127.5) + 127.5
g_img.numpy()

# Converter e exibir
img = array_to_img(g_img[0])
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis('off')
plt.title('Nova Face Gerada pela DCGAN', fontsize=14, fontweight='bold')
plt.savefig('/content/generated/new_image.png')
plt.show()

### 8.2 Galeria de Faces Geradas

In [ ]:
# Gerar múltiplas faces

num_faces = 16
noise_batch = tf.random.normal([num_faces, 100])

# Gerar batch de imagens
generated_images = dcgan.generator(noise_batch)
generated_images = (generated_images * 127.5) + 127.5
generated_images = generated_images.numpy()

# Plotar galeria
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
fig.suptitle('Faces Geradas', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    img = array_to_img(generated_images[i])
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f'Face {i+1}', fontsize=10)

plt.tight_layout()
plt.show()

## 9. Download dos Resultados

In [ ]:
# Compactar e fazer download dos resultados
print("Preparando arquivos para download...")

# Compactar imagens geradas
!zip -r /content/generated.zip /content/generated

# Download dos arquivos
files.download('/content/generated.zip')
files.download('generator.h5')